In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, cross_validate, learning_curve, RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, plot_confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


import os

In [ ]:
ds = pd.read_csv('data.csv', index_col=[0])
ds.drop(columns='Index', inplace=True)

In [ ]:
# are classes balanced?
print('class : count : percent')
print('0     : {}  : {:.2%}'.format(sum(ds['FLAG']==0), sum(ds['FLAG']==0)/len(ds['FLAG']) ))
print('1     : {}  : {:.2%}'.format(sum(ds['FLAG']==1), sum(ds['FLAG']==1)/len(ds['FLAG']) ))

sns.heatmap(ds.iloc[:,:1])
plt.show()

In [ ]:
print(ds.shape)
ds.sample(5)

In [ ]:
# column names
display(ds.columns)

# describtion of numeratic columns
display(ds.describe())

# Non-Null Count and type of columns
display(ds.info())

In [ ]:
# How many non-unique adresses?
_, uniq_idx, counts = np.unique(ds['Address'].to_numpy(), return_index=True, return_counts=True)
non_unique_addresses_idx = uniq_idx[counts > 1]
print("non-unique adresses count: {}".format(len(non_unique_addresses_idx), end='\n\n'))
# What are the flags of non-uniqe adresses
non_unique_addresses_flags = ds.iloc[non_unique_addresses_idx]['FLAG']
print("flags of non-unique adresses: ", end='')
print(*non_unique_addresses_flags)

ds.drop(columns='Address', inplace=True)

In [ ]:
display(np.unique(ds[' ERC20 most sent token type'].astype(str)))
display(np.unique(ds[' ERC20_most_rec_token_type'].astype(str)))

ds.drop(columns=[' ERC20 most sent token type', ' ERC20_most_rec_token_type'], inplace=True)

In [ ]:
#deteling columns that hold zero

ds.drop(columns=[' ERC20 avg time between rec tnx', ' ERC20 avg time between rec 2 tnx', ' ERC20 avg time between contract tnx',
                 ' ERC20 min val sent contract', ' ERC20 max val sent contract', ' ERC20 avg val sent contract', ' ERC20 avg time between sent tnx'], inplace=True)

In [ ]:
# missing values
missing_values = ds.isna()
missing_percent = missing_values.sum() / ds.shape[0] * 100
missing_df = pd.DataFrame([missing_values.sum(), missing_percent], ['count', 'percent'])
display(missing_df.sort_values(by='percent', axis=1, ascending=False))
missing_df.sort_values(by='percent', axis=1, ascending=False).to_csv('missing.csv')

sns.heatmap(missing_values, cbar=False, cmap='magma')
plt.show()

In [ ]:
non_fraud_rows, fraud_rows = np.where( [ds.iloc[:,0]==1] )
print(ds.iloc[fraud_rows,:].isna().sum()[-20:])

In [ ]:
missing_columns = ds.columns[ds.isna().sum() > 0]

In [ ]:
# correlation
corr = ds.corr()
plt.figure(figsize=(20,12))
sns.heatmap(np.abs(corr), cmap='coolwarm')
plt.show()